In [1]:
import numpy as np

In [2]:
"""
Define function for printing actual weight tables
"""

def print_weight_table(mantissa, actual):
    # Print header
    print(' '.join(['{:7}' for i in range(17)]).format(*np.insert(np.arange(-8,8), 0, 0)))
    # Print separation line
    print(''.join(['-' for i in range(8*17)]))
    # Print one row for every mantissa, iterate over columns (weight exponents)
    for i in range(actual.shape[1]):
        print(' '.join(['{:7}' for i in range(17)]).format(*np.insert(actual[:,i], 0, mantissa[i])))

In [3]:
"""
Define possible experiments
"""

# For each possible sign mode, define one experiment
experiments = {
    'excitatory': {
        'signMode': 'excitatory',
        'start': 0,
        'end': 255,
        'step': 1
    },
    'inhibitory': {
        'signMode': 'inhibitory',
        'start': -255,
        'end': 0,
        'step': 1
    },
    'mixed': {
        'signMode': 'mixed',
        'start': -256,
        'end': 254,
        'step': 2
    }
}

In [4]:
"""
Choose experiment
"""

exp = experiments['excitatory']
numWeightBits = 8

In [5]:
emulatedWeightsMantissa = np.arange(exp['start'], exp['end']+1, exp['step'])
emulatedWeightsActual = []

# Set isMixed to 1 if sign mode is mixed, else 0
isMixed = int(exp['signMode'] == 'mixed')

# Define number of available bits
numLsbBits = 8 - (numWeightBits - isMixed)

for w_exp in np.arange(-8,8):
    tmp = []
    for w in emulatedWeightsMantissa:
        # Shift weight mantissa
        w_shifted = int(np.floor((w / 2**numLsbBits))) * 2**numLsbBits
        #w_shifted = int((w / 2**numLsbBits)) * 2**numLsbBits
        # Scale weight with weight exponent
        w_scaled = w_shifted * 2 **(6.0+w_exp)
        # Shift scaled weight
        w_scaled_shifted = int(w_scaled / 2**6) * 2**6
        #w_scaled_shifted = int(np.floor(w_scaled / 2**6)) * 2**6
        # Apply 21 bit limit
        w = np.clip(w_scaled_shifted, -2097088, 2097088)

        tmp.append(w)

    emulatedWeightsActual.append(np.array(tmp))

# Transform to numpy array
emulatedWeightsActual = np.array(emulatedWeightsActual)

In [6]:
print_weight_table(emulatedWeightsMantissa, emulatedWeightsActual)

      0      -8      -7      -6      -5      -4      -3      -2      -1       0       1       2       3       4       5       6       7
----------------------------------------------------------------------------------------------------------------------------------------
      0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0       0
      1       0       0       0       0       0       0       0       0      64     128     256     512    1024    2048    4096    8192
      2       0       0       0       0       0       0       0      64     128     256     512    1024    2048    4096    8192   16384
      3       0       0       0       0       0       0       0      64     192     384     768    1536    3072    6144   12288   24576
      4       0       0       0       0       0       0      64     128     256     512    1024    2048    4096    8192   16384   32768
      5       0       0       0       0       0